In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import svd

In [5]:
from urllib.parse import urlparse, parse_qs

#if i want to include SMA/stock data for same company, need to scrape function not just symbol
def TimeSeriesDaily_df(stock_name):
    """
    Read a CSV file from a URL and return a DataFrame with symbol as the last column.
    """
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_name}&apikey=98WO4EK4CGOBTPQR&datatype=csv'
    df = pd.read_csv(url)
    df['symbol'] = stock_name
    return df
    

In [7]:
#19 home construction companies used in paper:
portfolio = ['FDX', 'UNP', 'KSU', 'NSC', 'UPS', 'CHRW', 'UAL', 'DAL', 'LUV', 'CSX', 'DHI', 'LEN', 'PHM', 'TOL', 'NVR', 'HD', 'LOW', 'SHW', 'SPY']

In [56]:
#would need to choose which data I want to use (open, high, low, close volumn) and it's easy to make a function that makes a series of this data
#Example: Close price
def get_close_price_matrices(portfolio, m):
    '''
    m is the number of days to look back
    '''
    data = []
    for s in portfolio:
        df = TimeSeriesDaily_df(s)
        prices = np.array(df['close'][:m][::-1])
        data.append(prices)
    M = np.array(data)
    M1 = M[:, :-1]
    M2 = M[:, 1:]
    return M1, M2


In [48]:
df = TimeSeriesDaily_df('FDX')
df.head()

,timestamp,open,high,low,close,volume,symbol
0,2025-05-21,219.720,220.3787,216.2900,216.81,1663529,FDX
1,2025-05-20,223.100,224.6700,220.0112,221.14,2437826,FDX
2,2025-05-19,225.800,227.1000,224.8100,226.12,1852487,FDX
3,2025-05-16,231.505,232.4900,226.6500,229.51,2271986,FDX
4,2025-05-15,232.150,234.0000,230.7700,231.11,2624906,FDX


In [57]:
def predict_price(M1, M2, t):
    ''' 
    t is the number of days in the future to predict
    ''' 
    M1_centered = M1 - np.mean(M1, axis=0, keepdims=True)
    U, s, Vt = np.linalg.svd(M1_centered, full_matrices=False)
    r = s.size
    Sigma_inv = np.diag(1.0/s)
    A_tilde = U.T @ M2 @ Vt.T @ Sigma_inv
    evals, evecs = np.linalg.eig(A_tilde)
    Phi = U @ evecs
    D = np.diag(np.exp(np.log(evals) * t))
    x1 = M1[:, 0]
    b = np.linalg.pinv(Phi) @ x1
    return Phi @ D @ b

In [59]:
M1, M2 = get_close_price_matrices(portfolio, 11)

KeyError: 'close'

In [60]:
aapl = TimeSeriesDaily_df('AAPL')
aapl

,{,symbol
0,"""Information"": ""We have detected your API ...",AAPL
1,},AAPL
